# NEP Gradient Analysis

This notebook shows how to extract and analyze gradients from `compute nep/gradient`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from parse_nep_gradients import NEPGradientParser

## 1. Setup - Define Model Dimensions

**IMPORTANT**: Set these to match your NEP model!

In [ ]:
# Replace with your actual model dimensions
NUM_NEURONS = 30
DESCRIPTOR_DIM = 50

# Initialize parser
parser = NEPGradientParser(NUM_NEURONS, DESCRIPTOR_DIM)
print(f"Expected gradient vector size: {parser.total_size}")

## 2. Load Gradient Data

In [ ]:
# Load data from LAMMPS output
data = parser.load_data('gradient.dat')
print(f"Loaded shape: {data.shape}")

## 3. Extract Components

In [ ]:
components = parser.extract_components(data)

print("Extracted components:")
print(f"  Timesteps: {components['timestep'].shape}")
print(f"  A_l (output weights): {components['A_l'].shape}")
print(f"  B_lj (hidden weights): {components['B_lj'].shape}")
print(f"  C_l (hidden biases): {components['C_l'].shape}")
print(f"  D (output bias): {components['D'].shape}")

## 4. Compute Statistics

In [ ]:
stats = parser.compute_statistics(components)

print(f"Gradient norm statistics:")
print(f"  Mean: {np.mean(stats['gradient_norm']):.6e}")
print(f"  Std:  {np.std(stats['gradient_norm']):.6e}")
print(f"  Min:  {np.min(stats['gradient_norm']):.6e}")
print(f"  Max:  {np.max(stats['gradient_norm']):.6e}")

## 5. Visualize Gradients

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
timesteps = components['timestep']

# Total gradient norm
axes[0, 0].plot(timesteps, stats['gradient_norm'], 'b-', linewidth=2)
axes[0, 0].set_xlabel('Timestep')
axes[0, 0].set_ylabel('Total Gradient Norm')
axes[0, 0].set_title('Total Gradient Magnitude')
axes[0, 0].grid(True, alpha=0.3)

# Component norms
axes[0, 1].plot(timesteps, stats['A_norm'], label='A_l (output weights)', linewidth=2)
axes[0, 1].plot(timesteps, stats['B_norm'], label='B_lj (hidden weights)', linewidth=2)
axes[0, 1].plot(timesteps, stats['C_norm'], label='C_l (hidden biases)', linewidth=2)
axes[0, 1].set_xlabel('Timestep')
axes[0, 1].set_ylabel('Component Norm')
axes[0, 1].set_title('Gradient Components')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# A_l heatmap
im = axes[1, 0].imshow(components['A_l'].T, aspect='auto', cmap='RdBu_r',
                       extent=[timesteps[0], timesteps[-1], 0, NUM_NEURONS])
axes[1, 0].set_xlabel('Timestep')
axes[1, 0].set_ylabel('Neuron Index')
axes[1, 0].set_title('A_l Evolution')
plt.colorbar(im, ax=axes[1, 0])

# Mean absolute values
axes[1, 1].semilogy(timesteps, stats['A_mean'], label='A_l', linewidth=2)
axes[1, 1].semilogy(timesteps, stats['B_mean'], label='B_lj', linewidth=2)
axes[1, 1].semilogy(timesteps, stats['C_mean'], label='C_l', linewidth=2)
axes[1, 1].set_xlabel('Timestep')
axes[1, 1].set_ylabel('Mean |Gradient|')
axes[1, 1].set_title('Mean Absolute Values (log scale)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Application: Uncertainty Quantification

In [ ]:
# Gradient norm as uncertainty metric
uncertainty = stats['gradient_norm']

# Find most uncertain configurations
n_uncertain = 10
most_uncertain_idx = np.argsort(uncertainty)[-n_uncertain:]

print(f"Top {n_uncertain} most uncertain timesteps:")
for i, idx in enumerate(most_uncertain_idx[::-1]):
    print(f"  {i+1}. Timestep {int(timesteps[idx])}: gradient norm = {uncertainty[idx]:.6e}")

## 7. Application: Parameter Sensitivity

In [ ]:
# Identify which parameters are most sensitive
A_l = components['A_l']
B_lj = components['B_lj']
C_l = components['C_l']

# Average absolute values across time
A_sensitivity = np.mean(np.abs(A_l), axis=0)
B_sensitivity = np.mean(np.abs(B_lj.reshape(-1, NUM_NEURONS * DESCRIPTOR_DIM)), axis=0)
C_sensitivity = np.mean(np.abs(C_l), axis=0)

# Plot sensitivity
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].bar(range(NUM_NEURONS), A_sensitivity)
axes[0].set_xlabel('Neuron')
axes[0].set_ylabel('Mean |∂E/∂v_l|')
axes[0].set_title('Output Weight Sensitivity')

axes[1].bar(range(NUM_NEURONS), C_sensitivity)
axes[1].set_xlabel('Neuron')
axes[1].set_ylabel('Mean |∂E/∂b_l|')
axes[1].set_title('Hidden Bias Sensitivity')

# B_lj as heatmap
B_sensitivity_2d = B_sensitivity.reshape(NUM_NEURONS, DESCRIPTOR_DIM)
im = axes[2].imshow(B_sensitivity_2d, aspect='auto', cmap='hot')
axes[2].set_xlabel('Descriptor')
axes[2].set_ylabel('Neuron')
axes[2].set_title('Hidden Weight Sensitivity')
plt.colorbar(im, ax=axes[2])

plt.tight_layout()
plt.show()

## 8. Export for Further Analysis

In [ ]:
# Save processed data
np.savez('gradient_analysis.npz',
         timesteps=timesteps,
         A_l=components['A_l'],
         B_lj=components['B_lj'],
         C_l=components['C_l'],
         D=components['D'],
         gradient_norm=stats['gradient_norm'])

print("Saved to gradient_analysis.npz")